# Guia práctica 2 

Utilizamos el dataset de kaggle : https://www.kaggle.com/datasets/andrewmvd/isic-2019/data


Instalar kaggle API y descargar el dataset mediante consola, escribir en consola :

pip install kaggle

kaggle datasets download -d andrewmvd/isic-2019

_________________________________________

Para establecer la configuracion de API de kaggle, es decir establecer donde se descarga el kaggle.json: 

In [3]:
import os

os.environ["KAGGLE_CONFIG_DIR"] = "C:/Users/elena/.kaggle"

Configuración del dispositivo (usa GPU si está disponible)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Descarga dataset

In [1]:
import kaggle

dataset_name = "andrewmvd/isic-2019"
kaggle.api.dataset_download_files(dataset_name, path="C:/Users/elena/Desktop/universidad/3º año/FSI/pythorch", unzip=True)


## Creacion pytorch dataset

---------------------------------------- Parte de la practica-----------------------------------------------------------

## Definicion de una red convolucional de neuronas, modelo (CNN)

Esta red neuronal tiene:

3 capas convulutivas cuyo proposito es extraer características de las imagenes:

- *conv1*:Toma imágenes de 3 canales (RGB) como entrada y produce 32 mapas de características.
- *conv2*: Toma los 32 mapas de características como entrada y produce 64 mapas de características.
- *conv3*: Toma los 64 mapas de características como entrada y produce 128 mapas de características.

1 capa de Max Pooling cuyo proposito es reducir el ancho y el alto de los mapas de caracteristicas para que disminuya la carga computacional y controlar el sobreajuste.Se aplica despues de cada capa convolucional.

2 capas de Dropout que se encarga de prevenir el sobreajuste al establecer aleatoriamente una fracción de las unidades de entrada en 0 durante el entrenamiento.
- dropout25: Aplicada después de conv2 y conv3 con una tasa de abandono del 0.25.
- dropout50: Aplicada después de la primera capa completamente conectada (fc1) con una tasa de abandono del 0.5.

2 Capas Completamente Conectadas (Densas) (Linear):

Propósito: Realizar la clasificación en base a las características extraídas por las capas convolucionales.
Detalles:
fc1: Una capa densa que aplana la salida de la última capa de agrupación en un solo vector y la reduce a 128 características.
fc2: La capa densa final que mapea estas 128 características a 8 unidades de salida 
Método de Avance (forward):

Propósito: Define la secuencia en la que los datos de entrada pasan a través de las capas.
Detalles: Los datos de entrada pasan por cada capa en el orden definido. Después de la última capa de agrupación, los datos se aplanan y pasan a través de las capas densas.
Tamaños de Entrada y Salida:

Tamaño de Entrada: El modelo espera imágenes de entrada de tamaño 224x224 después de la transformación RandomResizedCrop.
Tamaño de Salida: El modelo produce 10 clases (como se define en fc2).


In [ ]:


import torch.nn as nn
import torch.nn.functional as F


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)

        # Max pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Dropout layers
        self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)

        # Fully connected layers
        # ajustado para 150x150
        self.fc1 = nn.Linear(in_features=128 * 18 * 18, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=8)

    def forward(self, x):
        # Apply the first convolution, activation and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))

        # Apply the second convolution, activation, max pooling and dropout layers
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout25(x)

        # Apply the third convolution, activation, max pooling and dropout layers
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout25(x)

        # Flatten the output for the dense layer
        x = x.view(-1, 128 *18 * 18)

        # Apply the first dense layer with dropout
        x = F.relu(self.fc1(x))
        x = self.dropout50(x)

        # Apply the second dense layer (output layer)
        x = self.fc2(x)
        return x

# Create the modified model
model = CNN()
model.eval()  # Set the model to evaluation mode

# Print the modified model summary
model

##  Definir Transformaciones de Datos y Data Augmentation

Se aplican varias transformaciones a las imagenes de entrenamiento:
1. Se escala la imagen a 256x256
2. Rotacion aleatoria hasta 180 grados
3. Volteo horizontal aleatorio
4. Volteo vertical aleatorio
5. Recorte Aleatorio: Para simular el enfoque en diferentes partes de la lesión.
6. Ajustes de Color: Cambios sutiles en el brillo y el contraste  para simular variaciones de iluminación.


In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Rutas a los directorios de datos
train_dir = 'path/to/train_data'
valid_dir = 'path/to/valid_data'
test_dir = 'path/to/test_data'

# Carga el conjunto de datos de entrenamiento sin ninguna normalización para calcular la media y la desviación estándar
unnormalized_transform = transforms.Compose([
    transforms.Resize((150, 150)), 
    transforms.ToTensor()
])
unnormalized_dataset = datasets.ImageFolder(root=train_dir, transform=unnormalized_transform)
loader = DataLoader(unnormalized_dataset, batch_size=64, num_workers=0, shuffle=False)

# Calcular la media y la desviación estándar
mean_sum = torch.zeros(3)
std_sum = torch.zeros(3)
n_samples = 0

for images, _ in loader:
    images = images.view(images.size(0), 3, -1)
    mean, std = torch.std_mean(images, dim=[0, 2])
    mean_sum += mean * images.size(0)
    std_sum += std * images.size(0)
    n_samples += images.size(0)

mean = mean_sum / n_samples
std = std_sum / n_samples

# Transformaciones para el conjunto de entrenamiento
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),             
    transforms.RandomRotation(180),            
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.33)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# Transformaciones para el conjunto de validación y test (sin data augmentation)
test_valid_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# Parte de nelson (modificar aqui)
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=test_valid_transforms)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_valid_transforms)


## Configuración de DataSetLOaders


In [ ]:
# Crea los DataLoaders para cada conjunto de datos
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

## Early Stopping 

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

early_stopping = EarlyStopping(patience=5, min_delta=0.01)

# dentro del bucle de entrenamiento
for epoch in range(num_epochs):
    # Entrenamiento ...
    # Validación ...
    
    val_loss = # Calcula la pérdida de validación
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping")
        break